In [7]:
import pandas as pd
import numpy as np
import datetime
import lightgbm as lgb
import joblib
import graphviz 
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn import metrics
import os
os.environ["PATH"] += os.pathsep + 'C:/Program Files/Graphviz/bin/'


#this section is for import and initial data cleanup

otomoto=pd.read_csv("output_otomoto_18062022.csv", header=None, dtype='unicode') #we drop header to remove duplicate header lines and again align it as header
otomoto.replace(np.nan, -1, inplace=True)

otomoto.drop_duplicates(inplace=True)

new_header = otomoto.iloc[0] #grab the first row for the header
otomoto = otomoto[1:] #take the data below the header row
otomoto.columns = new_header #set the header row as the df header
#otomoto.info()
#otomoto.to_csv('ototmoto_test.csv')



otomoto["model"]=otomoto["model"].astype("str")+" ("+otomoto["brand"].astype("str")+")"
otomoto=otomoto.astype("category")
otomoto["price"]=otomoto["price"].astype("str")
otomoto["price"]=otomoto["price"].str.replace(" ", "")
otomoto["price"]=otomoto["price"].str.replace(",", ".")
otomoto["price"]=otomoto["price"].astype("float")
otomoto["mileage"]=otomoto["mileage"].astype("str")
otomoto["mileage"]=otomoto["mileage"].str.replace("no_mileage", "-1")
otomoto["mileage"]=otomoto["mileage"].str.replace(" ", "-1")
otomoto["mileage"]=otomoto["mileage"].astype("int")
otomoto["year"]=otomoto["year"].astype("int")
otomoto["cylinder_capacity"]=otomoto["cylinder_capacity"].astype("str")
otomoto["cylinder_capacity"]=otomoto["cylinder_capacity"].str.replace("no_cylinder_capacity", "-1")
otomoto["cylinder_capacity"]=otomoto["cylinder_capacity"].str.replace(" ","-1")
otomoto["cylinder_capacity"]=otomoto["cylinder_capacity"].astype("int")
otomoto["damaged"]=otomoto["damaged"].replace(-1,"Nie")
otomoto["power"]=otomoto["power"].astype("str")
otomoto["power"]=otomoto["power"].str.replace("no_power", "-1")
otomoto["power"]=otomoto["power"].str.replace(" ","")
otomoto["power"]=otomoto["power"].astype("int")
otomoto["year"]=otomoto["year"].astype("int")
otomoto["current_year"]=datetime.datetime.now().year
otomoto["age"]=otomoto["current_year"]-otomoto["year"]
otomoto["version"]=otomoto["version"].replace({-1: "Brak"})
otomoto["vin"]=otomoto["vin"].astype("str")
otomoto["country"]=otomoto["country"].replace({-1:"Brak"})
otomoto["no_crash"]=otomoto["no_crash"].replace(-1,"Nie")
otomoto["model"]=otomoto["model"].astype("category")
otomoto=otomoto.loc[~otomoto["title"].str.contains("cesja|leasing do oddania|odstapienie|przejecie|odstapie|odstepne")]
otomoto=otomoto.loc[otomoto["price_currency"]=="PLN"]

otomoto.info()
otomoto


<class 'pandas.core.frame.DataFrame'>
Int64Index: 157712 entries, 1 to 158848
Data columns (total 44 columns):
 #   Column                 Non-Null Count   Dtype   
---  ------                 --------------   -----   
 0   ad_id                  157712 non-null  category
 1   brand                  157712 non-null  category
 2   car_type               157712 non-null  category
 3   city                   157712 non-null  category
 4   city_name              157712 non-null  category
 5   co2                    157712 non-null  category
 6   color                  157712 non-null  category
 7   colour_type            157712 non-null  category
 8   condition              157712 non-null  category
 9   country                157712 non-null  category
 10  cylinder_capacity      157712 non-null  int32   
 11  damaged                157712 non-null  category
 12  district               157712 non-null  category
 13  doors                  157712 non-null  category
 14  drive               

,ad_id,brand,car_type,city,city_name,co2,color,colour_type,condition,country,...,title,transmission,update_date,url,user_id,version,vin,year,current_year,age
1,6097613608,Abarth,Auta miejskie,bydgoszcz,-1,-1,Czerwony,metallic,Używane,Polska,...,abarth grande punto abarth punto evo,Manualna,"21:06, 15 czerwca 2022",https://www.otomoto.pl/oferta/abarth-grande-pu...,no_user_id,Brak,ZFA19900000688048,2010,2022,12
2,6097570345,Abarth,Auta miejskie,niepolomice,-1,-1,Czerwony,metallic,Używane,Niemcy,...,abarth grande punto fiat grande punto abarth,Manualna,"15:24, 14 czerwca 2022",https://www.otomoto.pl/oferta/abarth-grande-pu...,no_user_id,Brak,ZFA19900000550106,2009,2022,13
3,6095558074,Abarth,Auta małe,tomaszow-mazowiecki,-1,-1,Inny kolor,no_colour_type,Używane,Szwajcaria,...,abarth 500,Manualna,"03:03, 14 czerwca 2022",https://www.otomoto.pl/oferta/abarth-500-ID6Ev...,13224624,Brak,xxxxxxxxxxxxxxxxx,2020,2022,2
4,6098004073,Abarth,Auta miejskie,olawa,-1,-1,Szary,no_colour_type,Używane,Brak,...,abarth 595 abarth 595 jak nowy automat zamiana...,Automatyczna,"19:29, 14 czerwca 2022",https://www.otomoto.pl/oferta/abarth-595-abart...,no_user_id,Brak,WBAXXXXXXXXXXXXXX,2018,2022,4
5,6098544832,Abarth,Auta miejskie,swiebodzin,-1,-1,Biały,metallic,Używane,Brak,...,abarth 1 4 benzyna 215 ps abarth gwarancja get...,Manualna,"11:40, 14 czerwca 2022",https://www.otomoto.pl/oferta/abarth-1-4-benzy...,10834783,Brak,ZFA19900000464246,2009,2022,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
158844,6098482995,Škoda,Sedan,warszawa,-1,-1,Srebrny,metallic,Używane,Polska,...,skoda superb skoda superb 2 0 tdi salon pl,Manualna,"17:48, 17 czerwca 2022",https://www.otomoto.pl/oferta/skoda-superb-sko...,no_user_id,Brak,TMBCR63U369116768,2006,2022,16
158845,6098482341,BMW,Sedan,warszawa,-1,-1,Biały,pearl,Używane,Stany Zjednoczone,...,bmw seria 3 bmw 3 f30 330i biala perla,Automatyczna,"18:14, 17 czerwca 2022",https://www.otomoto.pl/oferta/bmw-seria-3-bmw-...,no_user_id,330i Sport Line,WBA8B9C56HK884659,2017,2022,5
158846,6098483169,BMW,Kombi,zukow_60227,-1,-1,Czarny,pearl,Używane,Niemcy,...,bmw seria 5 superstan bogataopcja bezwkladufin...,Automatyczna,"18:09, 17 czerwca 2022",https://www.otomoto.pl/oferta/bmw-seria-5-supe...,no_user_id,525d Touring Sport-Aut,WBAMX11080DY09267,2012,2022,10
158847,6098483179,Volvo,Kombi,warszawa,-1,-1,Srebrny,metallic,Używane,Stany Zjednoczone,...,volvo xc 70 2 5 benz 210 km automat 4x4 zadban...,Automatyczna,"18:10, 17 czerwca 2022",https://www.otomoto.pl/oferta/volvo-xc-70-2-5-...,no_user_id,Brak,YV1SZ59H641169650,2004,2022,18


In [8]:
#new df based on otomoto

cars_dataset=otomoto[["price", "fuel", "power", "transmission", "mileage","year", "brand", "model", "no_crash", "damaged", "color", "country" ]].copy()
cars_dataset.sort_values(by=["year"], ascending=False, inplace=True)
cars_dataset
print(list(cars_dataset.year.unique()))
cars_dataset.info()
cars_dataset
x = cars_dataset.drop(['price'],axis=1)
y = cars_dataset.price
# train and test split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.33,random_state=42)
#my_df=pd.DataFrame(brand)
#my_df.to_csv("brand.csv")

[2023, 2022, 2021, 2020, 2019, 2018, 2017, 2016, 2015, 2014, 2013, 2012, 2011, 2010, 2009, 2008, 2007, 2006, 2005, 2004, 2003, 2002, 2001, 2000, 1999, 1998, 1997, 1996, 1995, 1994, 1993, 1992, 1991, 1990, 1989, 1988, 1987, 1986, 1985, 1984, 1983, 1982, 1981, 1980, 1979, 1978, 1977, 1976, 1975, 1974, 1973, 1972, 1971, 1970, 1969, 1968, 1967, 1966, 1965, 1964, 1963, 1962, 1961, 1960, 1959, 1958, 1957, 1956, 1955, 1954, 1953, 1952, 1951, 1950, 1949, 1948, 1947, 1946, 1939, 1938, 1937, 1936, 1934, 1932, 1929, 1928, 1927, 1926, 1925, 1923, 1920, 1917, 1915]
<class 'pandas.core.frame.DataFrame'>
Int64Index: 157712 entries, 7656 to 47944
Data columns (total 12 columns):
 #   Column        Non-Null Count   Dtype   
---  ------        --------------   -----   
 0   price         157712 non-null  float64 
 1   fuel          157712 non-null  category
 2   power         157712 non-null  int32   
 3   transmission  157712 non-null  category
 4   mileage       157712 non-null  int32   
 5   year    

,price,fuel,power,transmission,mileage,year,brand,model,no_crash,damaged,color,country
7656,734600.0,Benzyna,600,Automatyczna,5,2023,Audi,RS7 (Audi),Tak,Nie,Niebieski,Polska
32285,182900.0,Benzyna,180,Automatyczna,5,2023,Citroën,Inny (Citroën),Tak,Nie,Fioletowy,Polska
4104,773700.0,Benzyna,600,Automatyczna,10,2023,Audi,RS Q8 (Audi),Tak,Nie,Niebieski,Polska
4106,775800.0,Benzyna,600,Automatyczna,10,2023,Audi,RS6 (Audi),Tak,Nie,Szary,Polska
4473,845600.0,Benzyna,600,Automatyczna,5,2023,Audi,RS Q8 (Audi),Tak,Nie,Inny kolor,Polska
...,...,...,...,...,...,...,...,...,...,...,...,...
29623,110000.0,Benzyna,60,Manualna,60000,1925,Chevrolet,Inny (Chevrolet),Nie,Nie,Niebieski,Austria
34891,36900.0,Benzyna,25,Manualna,1,1923,Citroën,Inny (Citroën),Nie,Nie,Niebieski,Brak
47409,169000.0,Diesel,20,Manualna,1000,1920,Ford,Inny (Ford),Nie,Nie,Brązowy,Brak
52805,199900.0,Benzyna,20,Manualna,1,1917,Ford,Inny (Ford),Tak,Nie,Czerwony,Brak


In [5]:
#Train
# To define the input and output feature
#x = otomoto.drop(['price','url', 'number', 'year', 'current_year'],axis=1)
x = cars_dataset.drop(['price'],axis=1)
y = cars_dataset.price
# train and test split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.33,random_state=42)



model_lgbm = lgb.LGBMRegressor(bagging_freq=60, colsample_bytree=0.55, drop_rate=0.5, extra_trees=False, lambda_l1=0.6, lambda_l2=0.6, learning_rate=0.05,max_depth=8,n_estimators=364, num_leavers=95, subsample=0.65, random_state=42)
model_lgbm.fit(x_train,y_train, eval_set=[(x_test,y_test),(x_train,y_train)], verbose=20)
model_lgbm.booster_.save_model('lgbr_base.txt')
# save model
joblib.dump(model_lgbm, 'lgbr_model.pkl')

y_pred=model_lgbm.predict(x_test)


print('Training accuracy {:.4f}'.format(model_lgbm.score(x_train,y_train)))
print('Testing accuracy {:.4f}'.format(model_lgbm.score(x_test,y_test)))

ypred2=model_lgbm.predict(x) #predic values based on original data 
ypred2[0:10]  # showing first 5 predictions


fuel="Benzyna"
power=190
transmission="Automatyczna"
mileage=32000
year=2019
brand="Audi"
model="A5 (Audi)"
no_crash="Nie"
damaged="Nie"
color="Szary"
country="Austria"

df_pred=pd.DataFrame([[fuel,power,transmission,mileage, year, brand, model, no_crash,damaged, color, country ]], columns=['fuel','power','transmission','mileage', 'year', 'brand', 'model', 'no_crash','damaged', 'color', 'country'])
df_pred["fuel"]=df_pred["fuel"].astype("category")
df_pred["power"]=df_pred["power"].astype("int32" )
df_pred["transmission"]=df_pred["transmission"].astype("category")
df_pred["mileage"]=df_pred["mileage"].astype("int32")
df_pred["year"]=df_pred["year"].astype("int32")
df_pred["brand"]=df_pred["brand"].astype("category")
df_pred["model"]=df_pred["model"].astype("category")
df_pred["no_crash"]=df_pred["no_crash"].astype("category")
df_pred["damaged"]=df_pred["damaged"].astype("category")
df_pred["color"]=df_pred["color"].astype("category")
df_pred["country"]=df_pred["country"].astype("category")

#model = joblib.load('lgbr_model.pkl')
#model = lgb.load('lgbr_base.txt')
prediction = model_lgbm.predict(df_pred)
print(prediction)

d:\ProgramData\Anaconda3\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
d:\ProgramData\Anaconda3\lib\site-packages\lightgbm\basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
d:\ProgramData\Anaconda3\lib\site-packages\lightgbm\basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


[LightGBM] [Warning] Unknown parameter: num_leavers
[LightGBM] [Warning] lambda_l1 is set=0.6, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.6
[LightGBM] [Warning] lambda_l2 is set=0.6, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.6
[LightGBM] [Warning] bagging_freq is set=60, subsample_freq=0 will be ignored. Current value: bagging_freq=60
[20]	training's l2: 3.11089e+09	valid_0's l2: 2.86028e+09
[40]	training's l2: 1.52196e+09	valid_0's l2: 1.41489e+09
[60]	training's l2: 1.07423e+09	valid_0's l2: 1.04238e+09
[80]	training's l2: 8.63601e+08	valid_0's l2: 8.71379e+08
[100]	training's l2: 7.66255e+08	valid_0's l2: 7.8947e+08
[120]	training's l2: 7.10368e+08	valid_0's l2: 7.41584e+08
[140]	training's l2: 6.72893e+08	valid_0's l2: 7.17956e+08
[160]	training's l2: 6.47771e+08	valid_0's l2: 7.05706e+08
[180]	training's l2: 6.30562e+08	valid_0's l2: 6.98049e+08
[200]	training's l2: 6.12027e+08	valid_0's l2: 6.88223e+08
[220]	training's l2: 5.98112e+08	valid_0's l2:

In [ ]:
from lightgbm import LGBMRegressor
import scipy
import numpy as np
from sklearn.model_selection import ParameterSampler
from sklearn.model_selection import cross_val_score
import sklearn
from sklearn import datasets
import pandas as pd



X=cars_dataset.drop(['price'],axis=1)
y=cars_dataset.price

n_estimators=scipy.stats.randint(50,400)
max_depth=scipy.stats.randint(2,10)
lambda_l1=np.arange(0,0.8,0.05)
lambda_l2=np.arange(0,0.8,0.05)
extra_trees=[True,False]
subsample=np.arange(0.3,1.0,0.05)
bagging_freq=scipy.stats.randint(1,100)
colsample_bytree=np.arange(0.3,1.0,0.05)
num_leaves=scipy.stats.randint(10,100)
boosting=['gbdt','dart']
drop_rate=np.arange(0.1,0.8,0.1)
skip_drop=np.arange(0,0.7,0.1)
learning_rate=[0.00001,0.0001,0.001,0.01,0.015,0.02,0.05,0.1,0.15,0.2]
params = {'n_estimators':n_estimators,'max_depth':max_depth, 'subsample':subsample,
          'colsample_bytree':colsample_bytree,
          'learning_rate':learning_rate,
          'num_leaves':num_leaves,
          'boosting':boosting,
          'extra_trees':extra_trees,
          'lambda_l1':lambda_l1,
          'lambda_l2':lambda_l2,
          'bagging_freq':bagging_freq,
          'drop_rate':drop_rate}


params = list(ParameterSampler(params, n_iter=500))
print(len(params))
print(params)

store=[]
for i in range(len(params)):
    print(i)
    p=params[i]
    model_lgbm=LGBMRegressor(**p,objective='rmse')

    score=cross_val_score(model,X,y,cv=10)
    
    store.append({'parameters':p,'r2':score})
    
    
    print(score)
    print(i)
    
store=pd.DataFrame(store)
store.to_csv('LGBM_parameters_search.csv')

In [10]:
print('Training accuracy {:.4f}'.format(model_lgbm.score(x_train,y_train)))
print('Testing accuracy {:.4f}'.format(model_lgbm.score(x_test,y_test)))

Training accuracy 0.9430
Testing accuracy 0.9391


In [ ]:
lgb.plot_importance(model_lgbm)

In [ ]:
lgb.plot_metric(model_lgbm)

In [ ]:
lgb.plot_tree(model_lgbm, figsize=(200,200))

In [ ]:
metrics.plot_confusion_matrix(model_lgbm,x_test,y_pred)
#metrics.plot_confusion_matrix(model,x_test,y_test,cmap='Blues_r')

In [ ]:
#confusion_matrix(y_test, y_pred)
model.score(x_test, y_test)

 

In [ ]:
print(metrics.classification_report(y_test,model_lgbm.predict(x_test)))

In [11]:
ypred2=model_lgbm.predict(x) #predic values based on original data 
ypred2[0:10]  # showing first 5 predictions
cars_dataset["ypred2"]=ypred2
cars_dataset["url"]=otomoto["url"]
cars_dataset["diff"]=cars_dataset["ypred2"]-cars_dataset["price"]
cars_dataset.to_csv("model_resulst.csv")

In [ ]:
x_test[0:10]
#y_test

In [ ]:
y_test[0:10]

In [ ]:
print(metrics.r2_score(y_test, ypred2))
#print(metrics.mean_squared_log_error(y_test, ypred2))


In [ ]:
params.to_csv('LGBM_parameters.csv')

['lgbr_model.pkl']

In [64]:

fuel="Benzyna"
power=190
mileage=32000
transmission="Automatyczna"
year=2019
brand="Audi"
model="A5 (Audi)"
no_crash="Nie"
damaged="Nie"
color="Szary"
country="Austria"

df_pred=pd.DataFrame([[fuel,power,mileage, transmission, year, brand, model, no_crash,damaged, color, country ]], columns=['fuel','power','mileage', 'transmission', 'year', 'brand', 'model', 'no_crash','damaged', 'color', 'country'])
df_pred["fuel"]=df_pred["fuel"].astype("category")
df_pred["power"]=df_pred["power"].astype("int32" )
df_pred["mileage"]=df_pred["mileage"].astype("int32")
df_pred["transmission"]=df_pred["transmission"].astype("category")
df_pred["year"]=df_pred["year"].astype("int32")
df_pred["brand"]=df_pred["brand"].astype("category")
df_pred["model"]=df_pred["model"].astype("category")
df_pred["no_crash"]=df_pred["no_crash"].astype("category")
df_pred["damaged"]=df_pred["damaged"].astype("category")
df_pred["color"]=df_pred["color"].astype("category")
df_pred["country"]=df_pred["country"].astype("category")

#model = joblib.load('lgbr_model.pkl')
#model = lgb.load('lgbr_base.txt')
prediction = model.predict(df_pred)
prediction

AttributeError: 'str' object has no attribute 'predict'

AttributeError: 'str' object has no attribute 'predict'